In [1]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import pretty_midi
import warnings
import os
import joblib

In [2]:
labeled_features=np.load("labeled_features.npy")
label_list=np.load("label_list.npy")

In [3]:
# Shuffle Entire Dataset to Make Random
labeled_features = np.random.permutation(labeled_features)

# Partition into 3 Sets
num = len(labeled_features)
num_training = int(num * 0.6)
num_validation = int(num * 0.8)
training_data = labeled_features[:num_training]
validation_data = labeled_features[num_training:num_validation]
test_data = labeled_features[num_validation:]

# Separate Features from Labels
num_cols = training_data.shape[1] - 1
training_features = training_data[:, :num_cols]
validation_features = validation_data[:, :num_cols]
test_features = test_data[:, :num_cols]

# Format Features for Multi-class Classification
num_classes = len(label_list)
training_labels = training_data[:, num_cols].astype(int)
validation_labels = validation_data[:, num_cols].astype(int)
test_labels = test_data[:, num_cols].astype(int)

# Function for One-Hot Encoding
def one_hot(labels):
    return np.eye(num_classes)[labels].astype(int)

# Print to Check Dimentions and to Visualize
print(test_features[:10])
print(test_labels[:10])
print(one_hot(test_labels)[:10])

[[ 0.08414462  0.55        0.125       0.125     ]
 [ 0.34438873  0.55        0.125       0.125     ]
 [ 0.2267722  -0.35        0.125       0.125     ]
 [ 0.30181461  0.55        0.125       0.125     ]
 [-0.19512195 -0.35        0.125       0.125     ]
 [ 0.16176816  0.31        0.125       0.125     ]
 [ 0.28916243  0.55        0.125       0.125     ]
 [ 0.09452051  0.31        0.          0.125     ]
 [ 0.04817331  0.55        0.125       0.125     ]
 [ 0.17858251 -0.17        0.125       0.125     ]]
[1 8 1 1 2 7 8 1 8 1]
[[0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]]


In [4]:
def train_model(t_features, t_labels, v_features, v_labels):
    # Neural Network and SVM Configurations
    clf_1 = MLPClassifier(solver='adam', alpha=1e-4, hidden_layer_sizes=(5,), random_state=1)
    clf_2 = MLPClassifier(solver='adam', alpha=1e-4, hidden_layer_sizes=(5, 5), random_state=1)
    clf_3 = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(10, 10), random_state=1)
    clf_4 = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 100), random_state=1)
    clf_svm = SVC()
    
    # Keep Track of the Best Model
    best_clf = None
    best_accuracy = 0
    
    # Test the Accuracies of the Models and Get Best
    for clf in [clf_1, clf_2, clf_3, clf_4, clf_svm]:
        t_labels_hot = one_hot(t_labels)
        v_labels_hot = one_hot(v_labels)
        if (type(clf) == SVC):
            clf = clf.fit(t_features, t_labels)
        else:
            clf = clf.fit(t_features, t_labels_hot)
        predictions = clf.predict(v_features)
        count = 0
        for i in range(len(v_labels)):
            if (type(clf) != SVC):
                if np.array_equal(v_labels_hot[i], predictions[i]):
                    count += 1
            else:
                if v_labels[i] == predictions[i]:
                    count += 1
        accuracy = count / len(v_labels_hot)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_clf = clf

    print("Best Accuracy:", best_accuracy)
    return best_clf

classifier = train_model(training_features, training_labels, validation_features, validation_labels)
joblib.dump(classifier, 'classifier_model')

C:\Users\user\anaconda3\envs\midienv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\user\anaconda3\envs\midienv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\user\anaconda3\envs\midienv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Best Accuracy: 0.7235005658242173


['classifier_model']

In [5]:
def calculate_accuracy(clf, t_features, t_labels):
    count = 0
    predictions = clf.predict(t_features)
    t_labels_hot = one_hot(t_labels)
    for i in range(len(t_features)):
        if (type(clf) == SVC):
            if t_labels[i] == predictions[i]:
                count += 1
        else:
            if np.array_equal(t_labels_hot[i], predictions[i]):
                count += 1
    return count / len(t_features)

# Print the Test Accuracy
print(calculate_accuracy(classifier, test_features, test_labels))

0.7521689928328933
